In [1]:
import random

class Environment_3x3():
#     -------------
#     | A | B | C |
#     -------------
#     | D | E | F |
#     -------------
#     | G | H | I |
#     -------------
    
    def __init__(self):
        self.locationsCondition = {}
        self.locations_to_clean = []  # Necessary for goal_test function of Child Node
        self.locations = ['A','B','C', 'D','E','F', 'G','H','I']
        
        # Let's now instantiate each room randomly with Clean and Dirty
        for room in self.locations:
            condition = random.choice(['Clean', 'Dirty'])
            self.locationsCondition[room] = condition
        
        # Those locations which we have to clean
        for location, condition in self.locationsCondition.items():
            if condition == 'Dirty':
                self.locations_to_clean.append(location)    
                
    
    def print_environment(self):
        # Prints row wise
        rooms = 0
        for loc in self.locations:
            if rooms == 3:
                print()
                rooms = 0
            
            cond = self.locationsCondition[loc]
            if cond == 'Clean':
                cond = '_'
            elif cond == 'Dirty':
                cond = 'd'
            
            print('{}:{}'.format(loc, cond), end=' ')
            rooms += 1
        
        print()
        return

    
# This environment is when room B,E,H are Dirty, I have its tree in written form so this environment is for testing BFS Searching agent    
class Env:
    def __init__(self):
        # Let's now instantiate Rooms with 'Clean' or 'Dirty'
        self.locationsCondition = {}
        self.locations_to_clean = []
        self.locations = ['A','B','C', 'D','E','F', 'G','H','I']        
        
        for room in self.locations:
            if room == 'B' or room == 'E' or room == 'H':
                self.locationsCondition[room] = 'Dirty'
                continue
            self.locationsCondition[room] = 'Clean'
            
        for location, condition in self.locationsCondition.items():
            if condition == 'Dirty':
                self.locations_to_clean.append(location)
     
    
    def print_environment(self):
        rooms = 0
        for loc in self.locations:
            if rooms == 3:
                print()
                rooms = 0
            
            print('{}:{}'.format(loc, self.locationsCondition[loc]), end=' ')
            rooms += 1
        
        print()
        return                
                
        

In [2]:
class Node:
    def __init__(self, parent, state, parent_action, cleaned):
        self.parent = parent
        self.state = state
        self.parent_action = parent_action
        self.cleaned = cleaned   # A node will know that what locations parent, grandparent and great grandparents have already cleaned
         
            

class ChildNode(Node):
    def __init__(self, problem, parent, parent_action):
        state = problem.transition_model(parent, parent_action)  # This will give new state when a state applies an action
        cleaned = parent.cleaned.copy()   # Need to copy as no .copy() will give bugs because it would have same reference
        if parent_action == 'suck':   # This check helps in 'cleaned' of each node. If my parent's actions was suck so hence state has been cleaned so now my children should know too. Note: 'suck' on any state would give back that state but after that it can move up, down etc     
            cleaned.append(parent.state)
                                         
        super().__init__(parent=parent, 
                         state=state, 
                         parent_action=parent_action,
                         cleaned=cleaned)



# According to Book a problem for searching has:
# 1. initial state
# 2. possible actions
# 3. transition model (A description what each action does)
# 4. goal test (which determines that has goal been reached at given state)
# 5. path cost (that assigns numeric cost to each path)
class Problem:
    def __init__(self, Environment):
        self.initial_state = 'E'  # Can be on runtime too
        # State -Action-> New State
        self.transition_table = {   ('A','down'):'D',    
                                    ('A','right'):'B',
                                    ('A', 'suck'):'A',

                                    ('B','down'):'E',
                                    ('B','left'):'A',
                                    ('B','right'):'C',
                                    ('B', 'suck'):'B',

                                    ('C','down'):'F',
                                    ('C','left'):'B',
                                    ('C', 'suck'):'C',

                                    ('D','up'):'A',
                                    ('D','down'):'G',
                                    ('D','right'):'E',
                                    ('D', 'suck'):'D',

                                    ('E','up'):'B',
                                    ('E','down'):'H',
                                    ('E','left'):'D',
                                    ('E','right'):'F',
                                    ('E', 'suck'):'E',

                                    ('F','up'):'C',
                                    ('F','down'):'I',
                                    ('F','left'):'E',
                                    ('F', 'suck'):'F',

                                    ('G','up'):'D',
                                    ('G','right'):'H',
                                    ('G', 'suck'):'G',

                                    ('H','up'):'E',
                                    ('H','left'):'G',
                                    ('H','right'):'I',
                                    ('H', 'suck'):'H',

                                    ('I','up'):'F',
                                    ('I','left'):'H',
                                    ('I', 'suck'):'I'                      
                                                           }
        
        self.Environment = Environment   # Seemed better
        # This will tell us what actions are possible for any state Note: if a state is Dirty so it needs 'suck' action which is implemented in possible_actions method. Not here as it would make things messy
        self.possible_actions_table = {  'A':['right', 'down'],
                                         'B':['left', 'down', 'right'],
                                         'C':['left', 'down'],
                                         'D':['up', 'right', 'down'],
                                         'E':['up', 'left', 'right', 'down'],
                                         'F':['up', 'left', 'down'],
                                         'G':['up', 'right'],
                                         'H':['up', 'left', 'right'],
                                         'I':['up', 'left']                   }        
     
    
    # State -Action-> New state
    def transition_model(self, current_node, action):   
        return self.transition_table[(current_node.state, action)]        
    
    
   # Check whether goal has been achieved at this state (whether all rooms have been cleaned by parent, grandparent, great grandparents)
    def goal_test(self, current_node):
        cleaned = current_node.cleaned
        
        for loc_to_clean in self.Environment.locations_to_clean:
            if loc_to_clean not in cleaned:  # If there is a location that still needs to be cleaned return False else return True
                return False
        return True
    
    
    # Gives all possible actions for a state. It also handles 'suck' 
    # e,g for two calls this would give possible actions which would later result as part tree
    #     ..    X              If no need to suck then:         X
    #           | suck                                  left  / |right ..
    #           X                                            Y  Z ..
    #  .. left /  \right ..
    #       Y      ....
    def possible_actions(self, current_node):  
        if self.Environment.locationsCondition[current_node.state] == 'Dirty' and current_node.state not in current_node.cleaned:
            return ['suck']        
        
        return self.possible_actions_table[current_node.state] 

In [3]:
class SearchVacuumCleaner():
    def __init__(self, Problem):
        Problem.Environment.print_environment()
        
        self.Environment = Problem.Environment
        self.Problem = Problem        


    # What is my new location when this action is applied on current location
    def change_location(self, current_location, action):
        return self.Problem.transition_table[(current_location, action)]
    
    
    # Returns sequences of actions to take (by backtracking actions of state where goal test passed)
    def actions_to_take(self, current_node, actions_sequence=None):
        # First call 
        if actions_sequence == None:
            actions_sequence = []

        # If child..many recursive calls has resulted in Root of tree so its parent would be None. This is Base Case for stopping recursion    
        if current_node.parent is None:
            return actions_sequence
        
        # As first call to this method would be by leaf / state deep down
        actions_sequence.insert(0, current_node.parent_action)
       
        return self.actions_to_take(current_node.parent, actions_sequence)
      
        
    # Uses breadth first search for making a tree as state space and checking goal test on each node/state. Returns sequences of actions to take to reach goal (when all rooms are cleaned for Vacuum Cleaner agent)
    def bfs_goal(self):
        # Root node
        node = Node(parent=None, 
                    state=self.Problem.initial_state, 
                    parent_action=None,
                    cleaned=[])
        
        # If in Root node goal state has been achieved then return all sequences of actions
        if self.Problem.goal_test(node):
            return self.actions_to_take(node)
        
        frontier = [node]  # Or Queue. Frontier is the set of all leaf nodes available for expansion at any given point is called the frontier according to book
        
        while len(frontier) != 0:
            current_node = frontier.pop(0)
 
            # Make a child node from all possible actions on current node
            for action in self.Problem.possible_actions(current_node):    
                child = ChildNode(self.Problem, current_node, action)                           
                
                # If goal test has passed on child then return sequence of actions to take to reach to this state when goal has been achieved
                if self.Problem.goal_test(child) is True:
                    return self.actions_to_take(child)
                    
                frontier.append(child)
        
        return None  
    
    
    # Applying actions 
    def map_action_to_actuator(self, location, action):
        new_location = self.change_location(location, action)
        
        if action == 'up':
            print('🛸⬆️ Moving Upward to Location {}'.format(new_location))
            
        elif action == 'down':
            print('🛸⬇️ Moving Downward to Location {}'.format(new_location))

        elif action == 'left':
            print('🛸⬅️ Moving Left to Location {}'.format(new_location))
            
        elif action == 'right':
            print('🛸➡️ Moving Right to Location {}'.format(new_location))
            
        elif action == 'suck':
            print('🧹  Location {} is Dirty.\n    Location {} has been Cleaned.'.format( location, location ))            
            self.Environment.locationsCondition[location] = 'Clean'           
            
        
        return new_location
    
    
    def start(self):
        print('\n--- Vacuum started at location', self.Problem.initial_state, '---')
        current_location = self.Problem.initial_state
        actions_sequence = self.bfs_goal()  # Search where goal state has been achieved and then sequences of actions to take would be returned 

        if actions_sequence:   # If solution has been found by search algorithm             
            # Apply each action
            for action in actions_sequence:
                current_location = self.map_action_to_actuator(current_location, action)
        print()
        self.Environment.print_environment()
        print('--- All locations are Clean so goal state has reached ---')                    
        print('--- So Vacuum has stopped ---\n\n')

In [4]:
environment = Env()
problem = Problem(environment)
vacuum_agent = SearchVacuumCleaner(problem)
vacuum_agent.start()

A:Clean B:Dirty C:Clean 
D:Clean E:Dirty F:Clean 
G:Clean H:Dirty I:Clean 

--- Vacuum started at location E ---
🧹  Location E is Dirty.
    Location E has been Cleaned.
🛸⬆️ Moving Upward to Location B
🧹  Location B is Dirty.
    Location B has been Cleaned.
🛸⬇️ Moving Downward to Location E
🛸⬇️ Moving Downward to Location H
🧹  Location H is Dirty.
    Location H has been Cleaned.

A:Clean B:Clean C:Clean 
D:Clean E:Clean F:Clean 
G:Clean H:Clean I:Clean 
--- All locations are Clean so goal state has reached ---
--- So Vacuum has stopped ---




In [5]:
for i in range(0,100):
    environment = Environment_3x3()
    problem = Problem(environment)
    vacuum_agent = SearchVacuumCleaner(problem)
    vacuum_agent.start()

A:_ B:_ C:d 
D:_ E:d F:d 
G:_ H:d I:d 

--- Vacuum started at location E ---
🧹  Location E is Dirty.
    Location E has been Cleaned.
🛸⬇️ Moving Downward to Location H
🧹  Location H is Dirty.
    Location H has been Cleaned.
🛸➡️ Moving Right to Location I
🧹  Location I is Dirty.
    Location I has been Cleaned.
🛸⬆️ Moving Upward to Location F
🧹  Location F is Dirty.
    Location F has been Cleaned.
🛸⬆️ Moving Upward to Location C
🧹  Location C is Dirty.
    Location C has been Cleaned.

A:_ B:_ C:_ 
D:_ E:_ F:_ 
G:_ H:_ I:_ 
--- All locations are Clean so goal state has reached ---
--- So Vacuum has stopped ---


A:_ B:d C:_ 
D:_ E:_ F:_ 
G:_ H:d I:_ 

--- Vacuum started at location E ---
🛸⬆️ Moving Upward to Location B
🧹  Location B is Dirty.
    Location B has been Cleaned.
🛸⬇️ Moving Downward to Location E
🛸⬇️ Moving Downward to Location H
🧹  Location H is Dirty.
    Location H has been Cleaned.

A:_ B:_ C:_ 
D:_ E:_ F:_ 
G:_ H:_ I:_ 
--- All locations are Clean so goal state has re

🧹  Location E is Dirty.
    Location E has been Cleaned.
🛸⬆️ Moving Upward to Location B
🧹  Location B is Dirty.
    Location B has been Cleaned.
🛸⬅️ Moving Left to Location A
🧹  Location A is Dirty.
    Location A has been Cleaned.
🛸➡️ Moving Right to Location B
🛸➡️ Moving Right to Location C
🧹  Location C is Dirty.
    Location C has been Cleaned.
🛸⬇️ Moving Downward to Location F
🧹  Location F is Dirty.
    Location F has been Cleaned.
🛸⬇️ Moving Downward to Location I
🧹  Location I is Dirty.
    Location I has been Cleaned.
🛸⬅️ Moving Left to Location H
🧹  Location H is Dirty.
    Location H has been Cleaned.
🛸⬅️ Moving Left to Location G
🧹  Location G is Dirty.
    Location G has been Cleaned.

A:_ B:_ C:_ 
D:_ E:_ F:_ 
G:_ H:_ I:_ 
--- All locations are Clean so goal state has reached ---
--- So Vacuum has stopped ---


A:_ B:d C:d 
D:_ E:d F:d 
G:d H:_ I:_ 

--- Vacuum started at location E ---
🧹  Location E is Dirty.
    Location E has been Cleaned.
🛸⬆️ Moving Upward to Locatio

🛸⬆️ Moving Upward to Location B
🛸⬅️ Moving Left to Location A
🧹  Location A is Dirty.
    Location A has been Cleaned.
🛸⬇️ Moving Downward to Location D
🧹  Location D is Dirty.
    Location D has been Cleaned.
🛸⬇️ Moving Downward to Location G
🧹  Location G is Dirty.
    Location G has been Cleaned.
🛸➡️ Moving Right to Location H
🛸➡️ Moving Right to Location I
🧹  Location I is Dirty.
    Location I has been Cleaned.
🛸⬆️ Moving Upward to Location F
🧹  Location F is Dirty.
    Location F has been Cleaned.
🛸⬆️ Moving Upward to Location C
🧹  Location C is Dirty.
    Location C has been Cleaned.

A:_ B:_ C:_ 
D:_ E:_ F:_ 
G:_ H:_ I:_ 
--- All locations are Clean so goal state has reached ---
--- So Vacuum has stopped ---


A:d B:_ C:_ 
D:d E:d F:_ 
G:_ H:_ I:d 

--- Vacuum started at location E ---
🧹  Location E is Dirty.
    Location E has been Cleaned.
🛸⬆️ Moving Upward to Location B
🛸⬅️ Moving Left to Location A
🧹  Location A is Dirty.
    Location A has been Cleaned.
🛸⬇️ Moving Downward

🧹  Location E is Dirty.
    Location E has been Cleaned.
🛸⬆️ Moving Upward to Location B
🛸⬅️ Moving Left to Location A
🧹  Location A is Dirty.
    Location A has been Cleaned.
🛸⬇️ Moving Downward to Location D
🧹  Location D is Dirty.
    Location D has been Cleaned.
🛸⬇️ Moving Downward to Location G
🧹  Location G is Dirty.
    Location G has been Cleaned.
🛸➡️ Moving Right to Location H
🧹  Location H is Dirty.
    Location H has been Cleaned.
🛸➡️ Moving Right to Location I
🧹  Location I is Dirty.
    Location I has been Cleaned.
🛸⬆️ Moving Upward to Location F
🛸⬆️ Moving Upward to Location C
🧹  Location C is Dirty.
    Location C has been Cleaned.

A:_ B:_ C:_ 
D:_ E:_ F:_ 
G:_ H:_ I:_ 
--- All locations are Clean so goal state has reached ---
--- So Vacuum has stopped ---


A:d B:d C:_ 
D:d E:_ F:d 
G:_ H:_ I:d 

--- Vacuum started at location E ---
🛸⬆️ Moving Upward to Location B
🧹  Location B is Dirty.
    Location B has been Cleaned.
🛸⬅️ Moving Left to Location A
🧹  Location A is Di

🛸⬆️ Moving Upward to Location B
🧹  Location B is Dirty.
    Location B has been Cleaned.
🛸⬅️ Moving Left to Location A
🧹  Location A is Dirty.
    Location A has been Cleaned.
🛸➡️ Moving Right to Location B
🛸➡️ Moving Right to Location C
🧹  Location C is Dirty.
    Location C has been Cleaned.
🛸⬇️ Moving Downward to Location F
🧹  Location F is Dirty.
    Location F has been Cleaned.
🛸⬇️ Moving Downward to Location I
🧹  Location I is Dirty.
    Location I has been Cleaned.
🛸⬅️ Moving Left to Location H
🧹  Location H is Dirty.
    Location H has been Cleaned.
🛸⬅️ Moving Left to Location G
🧹  Location G is Dirty.
    Location G has been Cleaned.

A:_ B:_ C:_ 
D:_ E:_ F:_ 
G:_ H:_ I:_ 
--- All locations are Clean so goal state has reached ---
--- So Vacuum has stopped ---


A:_ B:_ C:_ 
D:_ E:_ F:_ 
G:_ H:d I:d 

--- Vacuum started at location E ---
🛸⬇️ Moving Downward to Location H
🧹  Location H is Dirty.
    Location H has been Cleaned.
🛸➡️ Moving Right to Location I
🧹  Location I is Dir

🧹  Location E is Dirty.
    Location E has been Cleaned.
🛸⬆️ Moving Upward to Location B
🧹  Location B is Dirty.
    Location B has been Cleaned.
🛸⬅️ Moving Left to Location A
🛸⬇️ Moving Downward to Location D
🧹  Location D is Dirty.
    Location D has been Cleaned.
🛸➡️ Moving Right to Location E
🛸➡️ Moving Right to Location F
🧹  Location F is Dirty.
    Location F has been Cleaned.
🛸⬇️ Moving Downward to Location I
🧹  Location I is Dirty.
    Location I has been Cleaned.
🛸⬅️ Moving Left to Location H
🧹  Location H is Dirty.
    Location H has been Cleaned.

A:_ B:_ C:_ 
D:_ E:_ F:_ 
G:_ H:_ I:_ 
--- All locations are Clean so goal state has reached ---
--- So Vacuum has stopped ---


A:d B:d C:_ 
D:d E:d F:_ 
G:d H:_ I:_ 

--- Vacuum started at location E ---
🧹  Location E is Dirty.
    Location E has been Cleaned.
🛸⬆️ Moving Upward to Location B
🧹  Location B is Dirty.
    Location B has been Cleaned.
🛸⬅️ Moving Left to Location A
🧹  Location A is Dirty.
    Location A has been Clean

🧹  Location E is Dirty.
    Location E has been Cleaned.
🛸⬇️ Moving Downward to Location H
🧹  Location H is Dirty.
    Location H has been Cleaned.
🛸⬆️ Moving Upward to Location E
🛸⬅️ Moving Left to Location D
🧹  Location D is Dirty.
    Location D has been Cleaned.
🛸⬆️ Moving Upward to Location A
🧹  Location A is Dirty.
    Location A has been Cleaned.
🛸➡️ Moving Right to Location B
🛸➡️ Moving Right to Location C
🧹  Location C is Dirty.
    Location C has been Cleaned.

A:_ B:_ C:_ 
D:_ E:_ F:_ 
G:_ H:_ I:_ 
--- All locations are Clean so goal state has reached ---
--- So Vacuum has stopped ---


A:_ B:d C:_ 
D:d E:_ F:_ 
G:_ H:_ I:_ 

--- Vacuum started at location E ---
🛸⬆️ Moving Upward to Location B
🧹  Location B is Dirty.
    Location B has been Cleaned.
🛸⬅️ Moving Left to Location A
🛸⬇️ Moving Downward to Location D
🧹  Location D is Dirty.
    Location D has been Cleaned.

A:_ B:_ C:_ 
D:_ E:_ F:_ 
G:_ H:_ I:_ 
--- All locations are Clean so goal state has reached ---
--- So Vac

# Now Adding Performance measure such as search cost and path cost to the Search Agent 

In [6]:
# NOW ADDING PERFORMANCE MEASURE SUCH AS SEARCH COST AND PATH COST


import numpy.random as rand

class Environment_3x3():
#     -------------
#     | A | B | C |
#     -------------
#     | D | E | F |
#     -------------
#     | G | H | I |
#     -------------
    
    def __init__(self):
        self.locationsCondition = {}
        self.locations_to_clean = []  # Necessary for goal_test function of Child Node
        self.locations = ['A','B','C', 'D','E','F', 'G','H','I']
        
        # Let's now instantiate each room randomly with Clean and Dirty, rach room can be Clean with probability 0.8 and can be Dirty with probability 0.2
        for room in self.locations:
            condition = rand.choice(['Clean', 'Dirty'], p=[0.8, 0.2])
            self.locationsCondition[room] = condition
        
        # Those locations which we have to clean
        for location, condition in self.locationsCondition.items():
            if condition == 'Dirty':
                self.locations_to_clean.append(location)    
                
    
    def print_environment(self):
        # Prints row wise
        rooms = 0
        for loc in self.locations:
            if rooms == 3:
                print()
                rooms = 0
            
            cond = self.locationsCondition[loc]
            if cond == 'Clean':
                cond = '_'
            elif cond == 'Dirty':
                cond = 'd'
            
            print('{}:{}'.format(loc, cond), end=' ')
            rooms += 1
        
        print()
        return


In [10]:
class Node:
    def __init__(self, parent, state, parent_action, cleaned, path_cost):
        self.parent = parent
        self.state = state
        self.parent_action = parent_action
        self.cleaned = cleaned   # A node will know that what locations parent, grandparent and great grandparents have already cleaned
        self.path_cost = path_cost  

class ChildNode(Node):
    def __init__(self, problem, parent, parent_action):
        state = problem.transition_model(parent, parent_action)  # This will give new state when a state applies an action
        cleaned = parent.cleaned.copy()   # Need to copy as no .copy() will give bugs because it would have same reference
        path_cost = parent.path_cost + problem.step_cost(parent.state, parent_action)  # This would sum of step costs of path at each individual state
        
        
        if parent_action == 'suck':   # This check helps in 'cleaned' of each node. If my parent's actions was suck so hence state has been cleaned so now my children should know too. Note: 'suck' on any state would give back that state but after that it can move up, down etc     
            cleaned.append(parent.state)
                                         
        super().__init__(parent=parent, 
                         state=state, 
                         parent_action=parent_action,
                         cleaned=cleaned,
                         path_cost=path_cost)



# According to Book a problem for searching has:
# 1. initial state
# 2. possible actions
# 3. transition model (A description what each action does)
# 4. goal test (which determines that has goal been reached at given state)
# 5. path cost (that assigns numeric cost to each path)
class Problem:
    def __init__(self, Environment):
        self.initial_state = 'E'  # Can be on runtime too
        # State -Action-> New State
        self.transition_table = {   ('A','down'):'D',    
                                    ('A','right'):'B',
                                    ('A', 'suck'):'A',

                                    ('B','down'):'E',
                                    ('B','left'):'A',
                                    ('B','right'):'C',
                                    ('B', 'suck'):'B',

                                    ('C','down'):'F',
                                    ('C','left'):'B',
                                    ('C', 'suck'):'C',

                                    ('D','up'):'A',
                                    ('D','down'):'G',
                                    ('D','right'):'E',
                                    ('D', 'suck'):'D',

                                    ('E','up'):'B',
                                    ('E','down'):'H',
                                    ('E','left'):'D',
                                    ('E','right'):'F',
                                    ('E', 'suck'):'E',

                                    ('F','up'):'C',
                                    ('F','down'):'I',
                                    ('F','left'):'E',
                                    ('F', 'suck'):'F',

                                    ('G','up'):'D',
                                    ('G','right'):'H',
                                    ('G', 'suck'):'G',

                                    ('H','up'):'E',
                                    ('H','left'):'G',
                                    ('H','right'):'I',
                                    ('H', 'suck'):'H',

                                    ('I','up'):'F',
                                    ('I','left'):'H',
                                    ('I', 'suck'):'I'                      
                                                           }
        
        self.Environment = Environment   # Seemed better
        # This will tell us what actions are possible for any state Note: if a state is Dirty so it needs 'suck' action which is implemented in possible_actions method. Not here as it would make things messy
        self.possible_actions_table = {  'A':['right', 'down'],
                                         'B':['left', 'down', 'right'],
                                         'C':['left', 'down'],
                                         'D':['up', 'right', 'down'],
                                         'E':['up', 'left', 'right', 'down'],
                                         'F':['up', 'left', 'down'],
                                         'G':['up', 'right'],
                                         'H':['up', 'left', 'right'],
                                         'I':['up', 'left']                   }        
     
    
    # State -Action-> New state
    def transition_model(self, current_node, action):   
        return self.transition_table[(current_node.state, action)]        
    
    
   # Check whether goal has been achieved at this state (whether all rooms have been cleaned by parent, grandparent, great grandparents)
    def goal_test(self, current_node):
        cleaned = current_node.cleaned
        
        for loc_to_clean in self.Environment.locations_to_clean:
            if loc_to_clean not in cleaned:  # If there is a location that still needs to be cleaned return False else return True
                return False
        return True
    
    
    # Gives all possible actions for a state. It also handles 'suck' 
    # e,g for two calls this would give possible actions which would later result as part tree
    #     ..    X              If no need to suck then:         X
    #           | suck                                  left  / |right ..
    #           X                                            Y  Z ..
    #  .. left /  \right ..
    #       Y      ....
    def possible_actions(self, current_node):  
        if self.Environment.locationsCondition[current_node.state] == 'Dirty' and current_node.state not in current_node.cleaned:
            return ['suck']        
        
        return self.possible_actions_table[current_node.state] 
    
    
    # This would give us step cost at a state given an action
    def step_cost(self, current_state, action):
        if action == 'suck':
            return 5  
        return 1  

In [11]:
class SearchVacuumCleaner():
    def __init__(self, Problem):
        Problem.Environment.print_environment()
        
        self.Environment = Problem.Environment
        self.Problem = Problem        


    # What is my new location when this action is applied on current location
    def change_location(self, current_location, action):
        return self.Problem.transition_table[(current_location, action)]
    
    
    # Returns sequences of actions to take (by backtracking actions of state where goal test passed)
    def actions_to_take(self, current_node, actions_sequence=None):
        # First call 
        if actions_sequence == None:
            actions_sequence = []

        # If child..many recursive calls has resulted in Root of tree so its parent would be None. This is Base Case for stopping recursion    
        if current_node.parent is None:
            return actions_sequence
        
        # As first call to this method would be by leaf / state deep down
        actions_sequence.insert(0, current_node.parent_action)
       
        return self.actions_to_take(current_node.parent, actions_sequence)
      
        
    # Uses breadth first search for making a tree as state space and checking goal test on each node/state. Returns sequences of actions to take to reach goal (when all rooms are cleaned for Vacuum Cleaner agent)
    def bfs_goal(self):
        # Root node
        node = Node(parent=None, 
                    state=self.Problem.initial_state, 
                    parent_action=None,
                    cleaned=[],
                    path_cost=0)
        search_cost = 1
        
        # If in Root node goal state has been achieved then return all sequences of actions
        if self.Problem.goal_test(node):
            return self.actions_to_take(node), node.path_cost, search_cost
        
        frontier = [node]  # Or Queue. Frontier is the set of all leaf nodes available for expansion at any given point is called the frontier according to book
        
        while len(frontier) != 0:
            current_node = frontier.pop(0)
             
            # Make a child node from all possible actions on current node
            for action in self.Problem.possible_actions(current_node):    
                child = ChildNode(self.Problem, current_node, action)                           
                search_cost += 1
                
                # If goal test has passed on child then return sequence of actions to take to reach to this state when goal has been achieved
                if self.Problem.goal_test(child) is True:
                    return self.actions_to_take(child), child.path_cost, search_cost
                    
                frontier.append(child)
        
        return None, None, None  
    
    
    # Applying actions 
    def map_action_to_actuator(self, location, action):
        new_location = self.change_location(location, action)
        
        if action == 'up':
            print('🛸⬆️ Moving Upward to Location {}'.format(new_location))
            
        elif action == 'down':
            print('🛸⬇️ Moving Downward to Location {}'.format(new_location))

        elif action == 'left':
            print('🛸⬅️ Moving Left to Location {}'.format(new_location))
            
        elif action == 'right':
            print('🛸➡️ Moving Right to Location {}'.format(new_location))
            
        elif action == 'suck':
            print('🧹  Location {} is Dirty.\n    Location {} has been Cleaned.'.format( location, location ))            
            self.Environment.locationsCondition[location] = 'Clean'            
        
        return new_location
    
    
    def start(self):
        print('\n--- Vacuum started at location', self.Problem.initial_state, '---')
        current_location = self.Problem.initial_state
        actions_sequence, path_cost, search_cost = self.bfs_goal()  # Search where goal state has been achieved and then sequences of actions to take would be returned 

        if actions_sequence:   # If solution has been found by search algorithm             
            # Apply each action
            for action in actions_sequence:
                current_location = self.map_action_to_actuator(current_location, action)
        
        print()
        self.Environment.print_environment()
        print('--- All locations are Clean so goal state has reached ---')                    
        print('Path cost:', path_cost)
        print('Search cost:', search_cost)
        print('--- Vacuum has stopped ---\n\n')

In [12]:
for i in range(0,100):
    environment = Environment_3x3()
    problem = Problem(environment)
    vacuum_agent = SearchVacuumCleaner(problem)
    vacuum_agent.start()

A:_ B:d C:_ 
D:_ E:d F:_ 
G:_ H:_ I:_ 

--- Vacuum started at location E ---
🧹  Location E is Dirty.
    Location E has been Cleaned.
🛸⬆️ Moving Upward to Location B
🧹  Location B is Dirty.
    Location B has been Cleaned.

A:_ B:_ C:_ 
D:_ E:_ F:_ 
G:_ H:_ I:_ 
--- All locations are Clean so goal state has reached ---
Path cost: 11
Search cost: 7
--- Vacuum has stopped ---


A:_ B:_ C:_ 
D:_ E:_ F:d 
G:_ H:_ I:_ 

--- Vacuum started at location E ---
🛸➡️ Moving Right to Location F
🧹  Location F is Dirty.
    Location F has been Cleaned.

A:_ B:_ C:_ 
D:_ E:_ F:_ 
G:_ H:_ I:_ 
--- All locations are Clean so goal state has reached ---
Path cost: 6
Search cost: 12
--- Vacuum has stopped ---


A:d B:d C:_ 
D:_ E:_ F:_ 
G:_ H:_ I:_ 

--- Vacuum started at location E ---
🛸⬆️ Moving Upward to Location B
🧹  Location B is Dirty.
    Location B has been Cleaned.
🛸⬅️ Moving Left to Location A
🧹  Location A is Dirty.
    Location A has been Cleaned.

A:_ B:_ C:_ 
D:_ E:_ F:_ 
G:_ H:_ I:_ 
--- All

A:d B:_ C:d 
D:_ E:_ F:_ 
G:d H:_ I:_ 

--- Vacuum started at location E ---
🛸⬆️ Moving Upward to Location B
🛸➡️ Moving Right to Location C
🧹  Location C is Dirty.
    Location C has been Cleaned.
🛸⬅️ Moving Left to Location B
🛸⬅️ Moving Left to Location A
🧹  Location A is Dirty.
    Location A has been Cleaned.
🛸⬇️ Moving Downward to Location D
🛸⬇️ Moving Downward to Location G
🧹  Location G is Dirty.
    Location G has been Cleaned.

A:_ B:_ C:_ 
D:_ E:_ F:_ 
G:_ H:_ I:_ 
--- All locations are Clean so goal state has reached ---
Path cost: 21
Search cost: 6202
--- Vacuum has stopped ---


A:_ B:_ C:_ 
D:d E:_ F:_ 
G:_ H:_ I:d 

--- Vacuum started at location E ---
🛸⬅️ Moving Left to Location D
🧹  Location D is Dirty.
    Location D has been Cleaned.
🛸➡️ Moving Right to Location E
🛸➡️ Moving Right to Location F
🛸⬇️ Moving Downward to Location I
🧹  Location I is Dirty.
    Location I has been Cleaned.

A:_ B:_ C:_ 
D:_ E:_ F:_ 
G:_ H:_ I:_ 
--- All locations are Clean so goal state has

G:_ H:_ I:_ 
--- All locations are Clean so goal state has reached ---
Path cost: 11
Search cost: 7
--- Vacuum has stopped ---


A:_ B:d C:_ 
D:_ E:_ F:d 
G:_ H:_ I:_ 

--- Vacuum started at location E ---
🛸⬆️ Moving Upward to Location B
🧹  Location B is Dirty.
    Location B has been Cleaned.
🛸⬇️ Moving Downward to Location E
🛸➡️ Moving Right to Location F
🧹  Location F is Dirty.
    Location F has been Cleaned.

A:_ B:_ C:_ 
D:_ E:_ F:_ 
G:_ H:_ I:_ 
--- All locations are Clean so goal state has reached ---
Path cost: 13
Search cost: 100
--- Vacuum has stopped ---


A:_ B:_ C:_ 
D:d E:_ F:_ 
G:d H:_ I:_ 

--- Vacuum started at location E ---
🛸⬅️ Moving Left to Location D
🧹  Location D is Dirty.
    Location D has been Cleaned.
🛸⬇️ Moving Downward to Location G
🧹  Location G is Dirty.
    Location G has been Cleaned.

A:_ B:_ C:_ 
D:_ E:_ F:_ 
G:_ H:_ I:_ 
--- All locations are Clean so goal state has reached ---
Path cost: 12
Search cost: 68
--- Vacuum has stopped ---


A:d B:_ C:_ 
